In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [2]:
# Change this according to your directory preferred setting
path_to_train = "./data/train_1"

# This event is in Train_1
event_prefix = "event000001000"

In [3]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN, AgglomerativeClustering

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.528], eps=0.0035, dz0 = -0.00070, stepdz = 0.00001, stepeps = 0.000005, num_loops=100):                        
        self.rz_scales=rz_scales
        self.epsilon = eps
        self.dz0 = dz0
        self.stepdz = stepdz
        self.stepeps = stepeps
        self.num_loops = num_loops
        
    # remove outliers
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    # not sure what this function does?
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    # standard scale our data
    def _preprocess(self, hits):
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    
    def _init(self,dfh):
        # create the radius of the hit from the coordinates
        dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['z'].values**2)
        
        # radius excluding z coordinate
        dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
        
        # create some other features
        dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
        dfh['z1'] = dfh['z'].values/dfh['rt'].values
        dfh['x2'] = 1/dfh['z1'].values
        
        # step params
        dz0 = self.dz0
        stepdz = self.stepdz
        stepeps = self.stepeps
        mm = 1
        
        # loop from 1 to 100
#         for ii in tqdm(range(100)):
        for ii in range(self.num_loops):
            mm = mm*(-1)
            dz = mm*(dz0+ii*stepdz)
            dfh['a1'] = dfh['a0'].values+dz*dfh['z'].values*np.sign(dfh['z'].values)
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            dfh['x1'] = dfh['a1'].values/dfh['z1'].values
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','x1','x2']].values)
            cx = np.array([1, 1, 0.75, 0.5, 0.5])
            dfs = np.multiply(dfs, cx)
            
            # cluster our hits
            clusters=DBSCAN(eps=self.epsilon+ii*stepeps,min_samples=1,metric='euclidean',n_jobs=1).fit(dfs).labels_            
    
            # if we are in first iteration add clusters to our hits
            if ii==0:
                dfh['s1'] = clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            
            # else update our hits conditionally, if it's a better fit
            else:
                # put our new clusters to another feature
                dfh['s2'] = clusters
                
                # get the count of those clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()
                
                # if our new clusters are bigger, but less than 20, use the new ones instead
                cond = np.where((dfh['N2'].values>dfh['N1'].values) & (dfh['N2'].values<20))
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1
                
                # write the new clusters back to our dataframe
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
        
        # return our clusters
        return dfh['s1'].values    
    
    def predict(self, hits):    
        # init our clusters
        self.clusters = self._init(hits) 
        
        # preprocess our data
        X = self._preprocess(hits) 
        
        # create our last clusterer
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7, metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
        
        # try agglomerative clustering to put remaining points into clusters of 3 or so?
        # use cosine affinity to cluster points with same direction rather than points that are close to each other
        unclustered_points = np.sum(self.clusters == 0)
        if unclustered_points > 3:
            cl = AgglomerativeClustering(n_clusters=unclustered_points // 3, affinity="cosine", linkage="ward")
        else:
            cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7, metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
            
        # labels = unique clusters
        labels = np.unique(self.clusters)
        
        # remove outliers
        self._eliminate_outliers(labels,X)
        
        # init n_labels
        n_labels = 0
        
        # now we loop through the points that haven't been assigned to a cluster and assign them with
        # HDBSCAN
        while n_labels < len(labels):
            n_labels = len(labels)
            max_len = np.max(self.clusters)
            mask = self.clusters == 0
            self.clusters[mask] = cl.fit_predict(X[mask])+max_len
            
        return self.clusters

In [4]:
path_to_test = "./data/test"
filename = "dbscan_outliers"
eps = 0.00375
dz0 = -0.000720
stepdz = 0.00001
stepeps = 0.000005

def one_loop(event_id):
    hits  = pd.read_csv(path_to_test + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(path_to_test + '/event%s-cells.csv'%event_id)
    print('Event ID: ', event_id)
                
    # Track pattern recognition 
    model = Clusterer(eps=eps, dz0=dz0, stepdz=stepdz, stepeps=stepeps)
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    one_submission.to_csv('./%09d.dbscan_outliers_agg.csv.gz'%int(event_id), index=False, compression='gzip')
            
    return one_submission

def create_test_submissions(path_to_test = "./data/test", start=0, end=125):
    event_ids = [ '%09d'%i for i in range(start,end) ]

    pool = Pool(processes=14)
    results = pool.map(one_loop, event_ids)
    pool.close()
    
    return results

In [5]:
_ = create_test_submissions(start=0)

Event ID:  000000003
Event ID:  000000000
Event ID:  000000039
Event ID:  000000030
Event ID:  000000015
Event ID:  000000021
Event ID:  000000033
Event ID:  000000027
Event ID:  000000006
Event ID:  000000018
Event ID:  000000009
Event ID:  000000012
Event ID:  000000036
Event ID:  000000024


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


Event ID:  000000016


Event ID:  000000034



...

/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


In [6]:
event_ids = [ i for i in range(0,125) ]
submissions = []
for i,event_id in enumerate(event_ids):
    submission  = pd.read_csv('./%09d.dbscan_outliers_agg.csv.gz'%event_id, compression='gzip')
    submissions.append(submission)

# Create submission file
submission = pd.concat(submissions, axis=0)
submission.to_csv('20180706_dbscan_outliers_agg.csv.gz', index=False, compression='gzip')
print(len(submission))

13741466


In [5]:
dataset_submissions = []
dataset_scores = []
for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=50, nevents=5):
    # Track pattern recognition
    model = Clusterer(eps=0.0037)
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    dataset_submissions.append(one_submission)

    # Score for the event
    score = score_event(truth, one_submission)
    dataset_scores.append(score)

    print("Score for event %d: %.8f" % (event_id, score))
print('Mean score: %.8f' % (np.mean(dataset_scores)))


Score for event 1020: 0.42136636



Score for event 1021: 0.39042686



Score for event 1022: 0.42344322



Score for event 1023: 0.39881523



Score for event 1024: 0.40869721
Mean score: 0.40854978
